# Import Required Libraries
Import necessary libraries such as pandas, numpy, matplotlib, and sklearn.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load Dataset
Load the ratings and movies datasets using pandas.

In [ ]:
# Load Dataset
ratings = pd.read_csv('path_to_ratings.csv')
movies = pd.read_csv('path_to_movies.csv')

# Display the first few rows of the datasets
display(ratings.head(), movies.head())

# Create User-Movie Matrix
Create a sparse user-movie matrix using the ratings data.

In [ ]:
# Create a sparse user-movie matrix
user_movie_matrix_sparse = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

# Display the shape of the user-movie matrix
print(user_movie_matrix_sparse.shape)

# Build Recommendation Model
Build a recommendation model using NearestNeighbors from sklearn.

In [ ]:
# Build Recommendation Model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_movie_matrix_sparse)

# Define Movie Recommendation Function
Define a function to generate movie recommendations for a given user.

In [ ]:
# Function to recommend movies
def recommend_movies(user_id, model_knn, user_movie_matrix_sparse, movies, top_n):
    distances, indices = model_knn.kneighbors(user_movie_matrix_sparse[user_id], n_neighbors=top_n + 1)
    similar_users = indices.flatten()[1:]  # Exclude the user itself
    
    movie_scores = np.zeros(user_movie_matrix_sparse.shape[1])
    for similar_user in similar_users:
        movie_scores += user_movie_matrix_sparse[similar_user].toarray().flatten()
    
    recommended_movies = np.argsort(-movie_scores)[:top_n]
    recommended_movies = pd.DataFrame(recommended_movies, columns=['movieId']).merge(movies, on='movieId')
    return recommended_movies[['movieId', 'title']].head(top_n)

# Example usage
target_user = 1  # Example user ID
top_n = 10  # Number of recommendations
recommendations = recommend_movies(target_user, model_knn, user_movie_matrix_sparse, movies, top_n)
display(recommendations)

# Evaluation Metrics
Define functions to calculate recall@k and precision@k.

In [ ]:
# Define Evaluation Metrics
def recall_at_k(recommended_movies, relevant_movies, k):
    recommended_top_k = recommended_movies[:k]
    relevant_and_recommended = set(recommended_top_k) & set(relevant_movies)
    return len(relevant_and_recommended) / len(relevant_movies) if len(relevant_movies) > 0 else 0

def precision_at_k(recommended_movies, relevant_movies, k):
    recommended_top_k = recommended_movies[:k]
    relevant_and_recommended = set(recommended_top_k) & set(relevant_movies)
    return len(relevant_and_recommended) / k if k > 0 else 0

# Calculate Metrics for Recommendations
Calculate recall and precision for the recommendations generated for a sample of users.

In [ ]:
# Example metrics calculation
user_ids = ratings['userId'].unique()[:100]  # Sample of 100 users
relevant_movies = [1, 2, 3]  # Example relevant movies
metrics = []

for user_id in user_ids:
    recommended_movies = recommend_movies(user_id, model_knn, user_movie_matrix_sparse, movies, top_n)['movieId'].tolist()
    recall = recall_at_k(recommended_movies, relevant_movies, top_n)
    precision = precision_at_k(recommended_movies, relevant_movies, top_n)
    metrics.append((recall, precision))

display(metrics)

# Plot Metrics
Plot the recall@k and precision@k metrics using matplotlib.

In [ ]:
# Plot Metrics
def plot_metrics(metrics):
    recalls, precisions = zip(*metrics)
    ks = range(1, len(metrics) + 1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(ks, recalls, label='Recall@K', marker='o')
    plt.plot(ks, precisions, label='Precision@K', marker='x')
    plt.title('Recall@K and Precision@K')
    plt.xlabel('K')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_metrics(metrics)